In [1]:

"""
Connect senior:Mainland
"""
import pandas as pd
import json
import requests
def ssql(sql):
    admin_token = '2cebeb5c7edac5ef0d7c5a640e69fc7d43bf2cd6a24ce43dfee3dd33662c0bab'
    url = 'http://sensor.wb-intra.com/api/sql/query?token=%s&project=production' % admin_token
    data = {'q': sql, 'format': 'json'}
    req = requests.post(url,data)
    req_dec = req.content.decode()
    try:
        req_json = json.loads('[' + req_dec.replace('\n', ',')[:-1] + ']')
        df_d_id = pd.DataFrame(req_json)
        return df_d_id
    except:
        print(req_dec)

In [3]:
df = ssql("""SELECT t2.*,
       t3.total_game_num,
       t2.game_num/t3.total_game_num AS p_
FROM
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'register'
     AND appId IN ('20014',
                   '30014')
     AND date BETWEEN '2020-10-20' AND current_date()- interval 1 DAY
   GROUP BY 1,
            2)t1
JOIN
  (SELECT date,distinct_id,
               playerCount,
               count(distinct_id)AS game_num
   FROM events
   WHERE event = 'gameStart'
     AND gameTypeId = 1800
     AND date BETWEEN '2020-10-20' AND current_date()- interval 1 DAY
   GROUP BY 1,
            2,
            3)t2 ON t1.date = t2.date
AND t1.distinct_id = t2.distinct_id
JOIN
  (SELECT date,distinct_id,
               count(distinct_id)AS total_game_num
   FROM events
   WHERE event = 'gameStart'
     AND gameTypeId = 1800
     AND date BETWEEN '2020-10-20' AND current_date()- interval 1 DAY
   GROUP BY 1,
            2)t3 ON t1.date = t3.date
AND t1.distinct_id = t3.distinct_id HAVING t2.game_num/t3.total_game_num >= 0.5
""")

In [4]:
df

,date,playercount,distinct_id,game_num,total_game_num,p_
0,2020-11-05,9,110332424,2,4,0.500
1,2020-11-02,10,111932513,11,17,0.647
2,2020-11-07,9,136008921,1,1,1.000
3,2020-10-23,10,119744766,9,11,0.818
4,2020-11-21,10,145690021,1,1,1.000
...,...,...,...,...,...,...
2105308,2020-11-05,10,122939366,1,1,1.000
2105309,2020-11-23,9,145246880,1,1,1.000
2105310,2020-11-20,5,124237268,3,6,0.500
2105311,2020-11-08,10,114107344,1,1,1.000


In [5]:
df.groupby(['date','playercount']).count()

distinct_id  game_num  total_game_num    p_
date       playercount                                             
2020-10-20 4                     46        46              46    46
           5                     41        41              41    41
           6                     40        40              40    40
           7                     38        38              38    38
           8                     66        66              66    66
...                             ...       ...             ...   ...
2020-12-03 6                   1105      1105            1105  1105
           7                   1278      1278            1278  1278
           8                   1651      1651            1651  1651
           9                   1945      1945            1945  1945
           10                  2942      2942            2942  2942

[315 rows x 4 columns]

In [6]:
df.groupby(['date','playercount']).count().to_clipboard()

In [10]:
df_old = ssql("""SELECT t5.date,
          t5.distinct_id
   FROM
     (SELECT date,distinct_id
      FROM events
      WHERE event = 'register'
        AND appId IN ('20014',
                      '30014')
        AND date BETWEEN '2020-10-20' AND current_date()- interval 2 DAY
      GROUP BY 1,
               2)t4
   RIGHT JOIN
     (SELECT date,distinct_id
      FROM events
      WHERE event = 'login'
        AND appId IN ('20014',
                      '30014')
        AND date BETWEEN '2020-10-20' AND current_date()- interval 2 DAY
      GROUP BY 1,
               2)t5 ON t4.date = t5.date
   AND t4.distinct_id = t5.distinct_id
   WHERE t4.distinct_id IS NULL
""")

In [11]:
df_old

,date,distinct_id
0,2020-11-26,119589679
1,2020-11-01,115051741
2,2020-11-09,134757983
3,2020-12-01,131714164
4,2020-11-16,123677721
...,...,...
8886570,2020-12-02,113618870
8886571,2020-11-06,106676964
8886572,2020-11-20,107803982
8886573,2020-11-23,148418661


In [9]:
df_old.groupby(['date','playercount']).count().to_clipboard()